# Importation des packages

In [1]:
import requests
from bs4 import BeautifulSoup, Tag, NavigableString
from loky import ProcessPoolExecutor
from urllib.error import HTTPError, URLError
from urllib.request import urlopen
import pickle

In [2]:
url = "https://www.vinted.fr/help"

response = requests.get(url)

### Récupérer toutes les urls des réponses : 

In [3]:
urls = []
href = (BeautifulSoup(response.text)
          .select("html body div.site div.container div.faq div.faq__categories a.faq__category"))
for ref in href :
    urls.append("https://www.vinted.fr" + ref.get('href'))
urls

['https://www.vinted.fr/help/16-comment-ca-marche',
 'https://www.vinted.fr/help/4-vendre',
 'https://www.vinted.fr/help/5-acheter',
 'https://www.vinted.fr/help/313-envoi',
 'https://www.vinted.fr/help/418-paiements-et-transferts',
 'https://www.vinted.fr/help/15-confiance-et-securite',
 'https://www.vinted.fr/help/392-mon-compte-parametres',
 'https://www.vinted.fr/help/383-communaute',
 'https://www.vinted.fr/help/103-se-connecter']

In [6]:
# with open('urls_nv1.txt', 'w') as f:
#     for url in urls:
#         f.write(f'{url}\n')

In [7]:
urls = open("urls_nv1.txt", "r")
urls = urls.read().split("\n")
urls = urls[:-1]
urls

['https://www.vinted.fr/help/16-comment-ca-marche',
 'https://www.vinted.fr/help/4-vendre',
 'https://www.vinted.fr/help/5-acheter',
 'https://www.vinted.fr/help/313-envoi',
 'https://www.vinted.fr/help/418-paiements-et-transferts',
 'https://www.vinted.fr/help/15-confiance-et-securite',
 'https://www.vinted.fr/help/392-mon-compte-parametres',
 'https://www.vinted.fr/help/383-communaute',
 'https://www.vinted.fr/help/103-se-connecter']

In [16]:
urls_nv_2 = []
for url in urls :
    response_2 = requests.get(url)
    href_2 = (BeautifulSoup(response_2.text)
              .select("html body div.site div.container div.faq a.c-cell__link"))
    for ref in href_2 :
        urls_nv_2.append("https://www.vinted.fr" + ref.get('href'))
urls_nv_2[0:3]

['https://www.vinted.fr/help/26-vendre-etape-par-etape',
 'https://www.vinted.fr/help/25-acheter-sur-vinted-toutes-les-etapes',
 'https://www.vinted.fr/help/373-vendre-sur-vinted-c-est-gratuit']

In [17]:
# with open('urls_nv2.txt', 'w') as f:
#     for url in urls_nv_2:
#         f.write(f'{url}\n')

In [2]:
urls_nv_2 = open("urls_nv2.txt", "r")
urls_nv_2 = urls_nv_2.read().split("\n")
urls_nv_2 = urls_nv_2[:-1]

In [3]:
len(urls_nv_2)

52

In [21]:
def recuperatin_url(urls) :
    urls_nv_3 = []
    for url in urls :
        response_3 = requests.get(url)
        href_3 = (BeautifulSoup(response_3.text)
                  .select("html body div.site div.container div.faq a.c-cell__link"))
        if href_3 != [] :
            for ref in href_3 :
                urls_nv_3.append("https://www.vinted.fr" + ref.get('href'))
        else : 
            urls_nv_3.append(url)
    return urls_nv_3

In [22]:
url_nv3 = recuperatin_url(urls_nv_2)
len(url_nv3)

216

In [24]:
# with open('urls_nv3.txt', 'w') as f:
#     for url in url_nv3:
#         f.write(f'{url}\n')

In [27]:
urls_nv3 = open("urls_nv3.txt", "r")
urls_nv3 = urls_nv3.read().split("\n")
urls_nv3 = urls_nv3[:-1]

In [9]:
# def recuperatin_url_niv_2(urls):
#     with ProcessPoolExecutor() as epool:
#         liste_urls_nv_2 = epool.map(recuperatin_url, urls)
#     return list(liste_urls_nv_2)

# recuperatin_url_niv_2(urls_nv_2)

In [27]:
url_nv4 = recuperatin_url(url_nv3)
len(url_nv4)

221

In [28]:
# with open('urls_nv4.txt', 'w') as f:
#     for url in url_nv4:
#         f.write(f'{url}\n')

In [2]:
url_nv4 = open("urls_nv4.txt", "r")
url_nv4 = url_nv4.read().split("\n")
url_nv4 = url_nv4[:-1]

In [3]:
print(len(url_nv4))
url_nv4[0:4]

221


['https://www.vinted.fr/help/26-vendre-etape-par-etape',
 'https://www.vinted.fr/help/25-acheter-sur-vinted-toutes-les-etapes',
 'https://www.vinted.fr/help/373-vendre-sur-vinted-c-est-gratuit',
 'https://www.vinted.fr/help/356-comment-partager-mon-retour-d-experience-sur-vinted']

### Récupérer le texte (les réponses) des urls :

In [3]:
def ValidTag(monTag):
    if type(monTag) == Tag and monTag.has_attr("class"):
        if ' '.join(monTag.attrs["class"]) in ['tab-content', 'nav-tabs-horizontal', '“table-wrapper”']:
            return False
    return True

In [4]:
def getSelectedText(monTag):
    texte = []
    for child in monTag.children :
        if type(child) == NavigableString :
            if (child.string).strip() != '' and (child.string).strip() != ' ' :
                texte.append( " " + (child.string).strip() )
                
        ## Récupération des tableaux
        if type(child) == Tag and child.has_attr("class") and ' '.join(child.attrs["class"]) in ['“table-wrapper”']:
            tables = child.findAll('table')


            l = []
            nbr_valeur_ligne = []
            colonne = []
            
            for i,tr in enumerate(tables[0].findAll('tr')):
                if 'rowspan' in str(tr) : 
                    valeur_depart = i 
                            #print(tr)
                l.append(tr.text.replace('\n', '|#|'))

            liste_entete = l[0].split('|#|')
            while '' in liste_entete : 
                liste_entete.remove('')
                    #print(liste_entete)
            a_change = []
            for i,td in enumerate(tables[0].findAll('td')) :
                if td.has_attr("rowspan") : 
                    num_colonne = i % len(liste_entete) 
                    nbr_fois_col = td.attrs['rowspan']
                    valeur_a_ajouter = td.text
                    a_change = [valeur_depart + o for o in range(2,int(nbr_fois_col)+1)]

            n = 1
            for mot in l[1::] : 
                n +=1
                ligne = mot.split('|#|')
                while '' in ligne : 
                    ligne.remove('')
                        #print(ligne)
                if n in a_change: 
                    ligne[num_colonne:num_colonne] = [valeur_a_ajouter]
                        #print(ligne)
                for i in range(0,len(liste_entete)) : 
                    texte.append( str(liste_entete[i]) +  " : " + str(ligne[i]) )
                texte.append( '------' )       
                
        
                
        if type(child) == Tag and ValidTag(child) and child.name != 'table':
            if getSelectedText(child).strip() != '' and getSelectedText(child).strip() != ' ' :
                texte.append( getSelectedText(child).strip() )
                
                
        if type(child) == Tag and child.name == 'table':
            #print([child])
            tables = child.find('table')
            #print(tables)

            l = []
            nbr_valeur_ligne = []
            colonne = []
            #print([child][0])
            for i,tr in enumerate([child][0].findAll('tr')):
                if 'rowspan' in str(tr) : 
                    valeur_depart = i 
                l.append(tr.text.replace('\n', '|#|'))

            liste_entete = l[0].split('|#|')
            while '' in liste_entete : 
                liste_entete.remove('')
            a_change = []
            for i,td in enumerate([child][0].findAll('td')) :
                if td.has_attr("rowspan") : 
                    num_colonne = i % len(liste_entete) 
                    nbr_fois_col = td.attrs['rowspan']
                    valeur_a_ajouter = td.text
                    a_change = [valeur_depart + o for o in range(2,int(nbr_fois_col)+1)]

            n = 1
            for mot in l[1::] : 
                n +=1
                ligne = mot.split('|#|')
                while '' in ligne : 
                    ligne.remove('')
                        #print(ligne)
                if n in a_change: 
                    ligne[num_colonne:num_colonne] = [valeur_a_ajouter]
                        #print(ligne)
                for i in range(0,len(liste_entete)) : 
                    texte.append( str(liste_entete[i]) +  " : " + str(ligne[i]) )
                texte.append( '------' )             
        
    texte_final = '\n'.join(texte)
    return texte_final

In [7]:
liste_textes = []
for url in url_nv4 :
    print(url)
    try :
        html = urlopen(url)
    except (HTTPError, URLError) as e :
        sys.exit(e)

    bsObj = BeautifulSoup(html, "lxml")
    maDiv = bsObj.find("div", class_="site").find("div", class_="body-content").find("div", class_="faq__question-answer faq-content")
    texte = getSelectedText(maDiv).strip().replace('\xa0', ' ').replace('\u200b', ' ').replace('\u2192', ' ').replace('\u2194', ' ').replace('\u2028', ' ')
    liste_textes.append(texte)
liste_textes[0:3]

https://www.vinted.fr/help/26-vendre-etape-par-etape
https://www.vinted.fr/help/25-acheter-sur-vinted-toutes-les-etapes
https://www.vinted.fr/help/373-vendre-sur-vinted-c-est-gratuit
https://www.vinted.fr/help/356-comment-partager-mon-retour-d-experience-sur-vinted
https://www.vinted.fr/help/265-comment-utiliser-le-centre-d-aide
https://www.vinted.fr/help/375-ajouter-un-article-etape-par-etape
https://www.vinted.fr/help/26-vendre-etape-par-etape
https://www.vinted.fr/help/373-vendre-sur-vinted-c-est-gratuit
https://www.vinted.fr/help/52-qu-est-ce-que-je-peux-vendre-sur-vinted
https://www.vinted.fr/help/220-comment-echanger-un-article
https://www.vinted.fr/help/375-ajouter-un-article-etape-par-etape
https://www.vinted.fr/help/601-quelles-photos-dois-je-mettre-en-ligne-pour-mes-articles-de-marque
https://www.vinted.fr/help/48-comment-prendre-de-bonnes-photos
https://www.vinted.fr/help/49-comment-bien-decrire-mon-article
https://www.vinted.fr/help/50-comment-choisir-l-etat-de-mon-article


['Vendre, étape par étape\nVendre un article, c’est simple et gratuit ! Voilà comment faire :\nMettre en ligne un article.\nAjoute de\nbelles photos\n de l’article, écris une\ndescription\n détaillée,\nfixe le prix\n de ton article, choisis une\ntaille pour le colis\n .\nLa mise en ligne d’un article est gratuite.\nMettre en ligne plus d’un article aide à vendre plus vite.\nConfigure ton profil.\nParamètre ton\nporte-monnaie Vinted\n et fais\nvérifier ton compte\n .\nSélectionne le transport et les modes de paiement dans tes paramètres. Tu peux également faire une promotion sur ton catalogue.\nEn ajoutant une\ndescription et une photo sur ton profil\n , tu renforceras la confiance des autres membres.\nAttends les acheteurs.\nSois prêt à répondre aux questions des acheteurs.\nClique sur le bouton\nFaire une offre\n si tu veux proposer un meilleur prix.\nTu seras notifié durant tout le processus d’achat de ton acheteur.\nArticle vendu : il est temps de l’envoyer.\nEmballe ton article\n (

In [14]:
for elem in liste_textes[4:5]:
     print(elem.replace('  ','\n'))

In [25]:
#liste_textes[99].split('\n')

In [8]:
# with open('Reponses_aux_questions_A_laligne_3.txt', 'w') as f:
#     for texte in liste_textes:
#         f.write(f'{str(texte)}|#|')

#### EXEMPLE : 

In [20]:
liste_textes = []
for url in url_nv4[96:97] :
    print(url)
    try :
        html = urlopen(url)
    except (HTTPError, URLError) as e :
        sys.exit(e)

    bsObj = BeautifulSoup(html, "lxml")
    maDiv = bsObj.find("div", class_="site").find("div", class_="body-content").find("div", class_="faq__question-answer faq-content")
    texte = getSelectedText(maDiv).strip().replace('\xa0', ' ').replace('\u200b', ' ').replace('\u2192', ' ').replace('\u2194', ' ').replace('\u2028', ' ')
    liste_textes.append(texte)
liste_textes[0:3]

https://www.vinted.fr/help/441-options-d-envoi-pour-les-vendeurs


["Options d'envoi pour les vendeurs\nPour vendre plus rapidement, nous te conseillons d’offrir à tes acheteurs le plus d’options d’envoi possible. Cependant, assure-toi de proposer uniquement celles qui sont pratiques pour toi.\nLes modes d’envoi que tu peux offrir à tes acheteurs :\nRemarque :\n Pour les noms de pays, nous utilisons les abréviations suivantes : FR - France; ES - Espagne; BE - Belgique; LU - Luxembourg; NL - Pays-Bas.\nTransporteur : Mondial Relay\nSuivi : Oui\nMode d'envoi : De / à point relais\nPays : FR   FR; FR   ES; BE  BE; ES   ES; FR   BE; FR   LU; BE   BE; BE   LU\n------\nTransporteur : Chrono Shop2Shop\nSuivi : Oui\nMode d'envoi : De / à point relais\nPays : FR   FR; FR   LU; FR   BE; FR   NL \n------\nTransporteur : La Poste\nSuivi : Oui\nMode d'envoi : D'un bureau de poste à l'adresse de l'acheteur\nPays : FR   FR \n------\nTransporteur : Colissimo\nSuivi : Oui\nMode d'envoi : D'un bureau de poste à l'adresse de l'acheteur\nPays : FR   BE; FR   ES; FR   NL 

In [21]:
for elem in liste_textes[0].split('\n') :
    print(elem)

Options d'envoi pour les vendeurs
Pour vendre plus rapidement, nous te conseillons d’offrir à tes acheteurs le plus d’options d’envoi possible. Cependant, assure-toi de proposer uniquement celles qui sont pratiques pour toi.
Les modes d’envoi que tu peux offrir à tes acheteurs :
Remarque :
 Pour les noms de pays, nous utilisons les abréviations suivantes : FR - France; ES - Espagne; BE - Belgique; LU - Luxembourg; NL - Pays-Bas.
Transporteur : Mondial Relay
Suivi : Oui
Mode d'envoi : De / à point relais
Pays : FR   FR; FR   ES; BE  BE; ES   ES; FR   BE; FR   LU; BE   BE; BE   LU
------
Transporteur : Chrono Shop2Shop
Suivi : Oui
Mode d'envoi : De / à point relais
Pays : FR   FR; FR   LU; FR   BE; FR   NL 
------
Transporteur : La Poste
Suivi : Oui
Mode d'envoi : D'un bureau de poste à l'adresse de l'acheteur
Pays : FR   FR 
------
Transporteur : Colissimo
Suivi : Oui
Mode d'envoi : D'un bureau de poste à l'adresse de l'acheteur
Pays : FR   BE; FR   ES; FR   NL 
------
Transporteur : Bp

### Récupérer toutes les questions : 

In [4]:
url = "https://www.vinted.fr/help"
response_2 = requests.get(url)

question = (BeautifulSoup(response_2.text)
          .select("html body div.site div.container div.faq div.faq__categories  div.faq__category-title"))

liste_categories = []
for i in range(0,len(question)) :
    liste_categories.append( question[i].text.strip() )
liste_categories

['Comment ça marche',
 'Vendre',
 'Acheter',
 'Envoi',
 'Paiements et transferts',
 'Confiance et sécurité',
 'Mon compte & Paramètres',
 'Communauté',
 'Se connecter']

In [5]:
dico_categories = {}
for i in range(0,len(question)) :
    dico_categories[ question[i].text.strip() ] = []
dico_categories

{'Comment ça marche': [],
 'Vendre': [],
 'Acheter': [],
 'Envoi': [],
 'Paiements et transferts': [],
 'Confiance et sécurité': [],
 'Mon compte & Paramètres': [],
 'Communauté': [],
 'Se connecter': []}

In [31]:
liste_categories[0]

'Comment ça marche'

In [38]:
urls

['https://www.vinted.fr/help/16-comment-ca-marche',
 'https://www.vinted.fr/help/4-vendre',
 'https://www.vinted.fr/help/5-acheter',
 'https://www.vinted.fr/help/313-envoi',
 'https://www.vinted.fr/help/418-paiements-et-transferts',
 'https://www.vinted.fr/help/15-confiance-et-securite',
 'https://www.vinted.fr/help/392-mon-compte-parametres',
 'https://www.vinted.fr/help/383-communaute',
 'https://www.vinted.fr/help/103-se-connecter']

In [8]:
for num_url in range(0,len(urls)) :
    categorie_nv2= []
    response_nv2 = requests.get(urls[num_url])
    cat_nv2 = (BeautifulSoup(response_nv2.text).select("html body div.site div.container div.faq div.c-cell__title"))
    for elem in range(0,len(cat_nv2)) :
        dico_categories[liste_categories[num_url]].append(cat_nv2[elem].text.strip())
dico_categories

{'Comment ça marche': ['Vendre, étape par étape',
  'Acheter sur Vinted : toutes les étapes',
  "Vendre sur Vinted, c'est gratuit ?",
  'Comment partager mon retour d’expérience sur Vinted ?',
  "Comment utiliser le Centre d'aide ?"],
 'Vendre': ['Vendre sur Vinted',
  'Télécharger un article',
  'Vendre plus vite',
  'Envoyer un article',
  'Recevoir son paiement',
  'Évaluations & Communication',
  'Retours et annulations',
  'Mes articles'],
 'Acheter': ["Processus d'achat",
  'Trouver un article',
  'Acheter un article',
  'Faire un paiement',
  'Envoi d’un article',
  'Évaluations & Communication',
  'Remboursements et annulations'],
 'Envoi': ['Politique de remboursement de Vinted',
  'Je suis vendeur',
  'Je suis acheteur'],
 'Paiements et transferts': ['Méthodes de paiement',
  'Porte-monnaie Vinted',
  'Versements'],
 'Confiance et sécurité': ['Protection acheteurs',
  'Protection vendeurs',
  'Sécurité et sérénité',
  'Utilisation et protection des données personnelles',
  'V

#### Concaténer les catégories/questions ensemble :

In [9]:
liste_categories = []
for key, categorie in dico_categories.items():
    for value in categorie :
        liste_categories.append(key + ' ' + value)
liste_categories

['Comment ça marche Vendre, étape par étape',
 'Comment ça marche Acheter sur Vinted : toutes les étapes',
 "Comment ça marche Vendre sur Vinted, c'est gratuit ?",
 'Comment ça marche Comment partager mon retour d’expérience sur Vinted ?',
 "Comment ça marche Comment utiliser le Centre d'aide ?",
 'Vendre Vendre sur Vinted',
 'Vendre Télécharger un article',
 'Vendre Vendre plus vite',
 'Vendre Envoyer un article',
 'Vendre Recevoir son paiement',
 'Vendre Évaluations & Communication',
 'Vendre Retours et annulations',
 'Vendre Mes articles',
 "Acheter Processus d'achat",
 'Acheter Trouver un article',
 'Acheter Acheter un article',
 'Acheter Faire un paiement',
 'Acheter Envoi d’un article',
 'Acheter Évaluations & Communication',
 'Acheter Remboursements et annulations',
 'Envoi Politique de remboursement de Vinted',
 'Envoi Je suis vendeur',
 'Envoi Je suis acheteur',
 'Paiements et transferts Méthodes de paiement',
 'Paiements et transferts Porte-monnaie Vinted',
 'Paiements et tra

In [10]:
# with open('concat_categories_nv2.txt', 'w') as f:
#     for question in liste_categories:
#         f.write(f'{str(question)}|#|')

In [11]:
liste_categories = open("concat_categories_nv2.txt", "r")
liste_categories = liste_categories.read().split("|#|")
liste_categories = liste_categories[:-1]

## Questions niveau 2 :

In [17]:
liste_concat2 =[]
num=0

for url in urls_nv_2 : 
    response_2 = requests.get(url)
    question = (BeautifulSoup(response_2.text)
          .select("html body div.site div.container div.faq div.c-cell__title"))
    if question != [] :
        for i in range(0,len(question)) :
            liste_concat2.append( liste_categories[num] + ' ' + question[i].text.strip() )
    else:
        liste_concat2.append( liste_categories[num] )
    print(num)
    num+=1

In [36]:
def concat_question(urls_nv_, liste_categories):
    liste_concat2 =[]
    num=0

    for url in urls_nv_ : 
            response_2 = requests.get(url)
            question = (BeautifulSoup(response_2.text)
                  .select("html body div.site div.container div.faq div.c-cell__title"))
            if question != [] :
                for i in range(0,len(question)) :
                    liste_concat2.append( liste_categories[num] + ' ' + question[i].text.strip() )
            else:
                liste_concat2.append( liste_categories[num] )
            #print(num)
            num+=1
    return liste_concat2

In [37]:
liste_concat2 = concat_question(urls_nv_2, liste_categories)

In [22]:
len(liste_categories)

52

In [15]:
len(liste_concat2)

216

In [16]:
# with open('concat_categories_nv3.txt', 'w') as f:
#     for question in liste_concat2:
#         f.write(f'{str(question)}|#|')

In [18]:
liste_concat2 = open("concat_categories_nv3.txt", "r")
liste_concat2 = liste_concat2.read().split("|#|")
liste_concat2 = liste_concat2[:-1]

In [19]:
len(liste_categories)

52

## Question niveau 3 :

In [38]:
liste_concat3 = concat_question(urls_nv3, liste_concat2)
liste_concat3

['Comment ça marche Vendre, étape par étape',
 'Comment ça marche Acheter sur Vinted : toutes les étapes',
 "Comment ça marche Vendre sur Vinted, c'est gratuit ?",
 'Comment ça marche Comment partager mon retour d’expérience sur Vinted ?',
 "Comment ça marche Comment utiliser le Centre d'aide ?",
 'Vendre Vendre sur Vinted Ajouter un article étape par étape',
 'Vendre Vendre sur Vinted Vendre, étape par étape',
 "Vendre Vendre sur Vinted Vendre sur Vinted, c'est gratuit ?",
 "Vendre Vendre sur Vinted Qu'est-ce que je peux vendre sur Vinted ?",
 'Vendre Vendre sur Vinted Comment échanger un article',
 'Vendre Télécharger un article Ajouter un article étape par étape',
 'Vendre Télécharger un article Quelles photos dois-je mettre en ligne pour mes articles de marque ?',
 'Vendre Télécharger un article Comment prendre de bonnes photos ?',
 'Vendre Télécharger un article Comment bien décrire mon article ?',
 "Vendre Télécharger un article Comment choisir l'état de mon article ?",
 'Vendre 

In [32]:
print(len(urls_nv3))
len(urls_nv_2)

216


52

In [39]:
len(liste_concat3)

221

In [ ]:
# with open('concat_categories_nv4.txt', 'w') as f:
#     for question in liste_concat3:
#         f.write(f'{str(question)}|#|')

In [2]:
liste_concat3 = open("concat_categories_nv4.txt", "r")
liste_concat3 = liste_concat3.read().split("|#|")
liste_concat3 = liste_concat3[:-1]

In [2]:
len(liste_concat3)

221